In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q transformers datasets accelerate peft bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.0 MB/s eta 0:00:00


In [3]:
model_id = "mistralai/Mistral-7B-v0.1"
data_path = "/content/drive/MyDrive/qna_finetune.jsonl"
output_dir = "/content/drive/MyDrive/stackbot/models/mistral-lora"


In [4]:
import pandas as pd
from datasets import Dataset

# Step 1: Load JSONL manually using pandas
df = pd.read_json("/content/drive/MyDrive/qna_finetune.jsonl", lines=True)

# Step 2: Format it in Mistral's prompt style
df["text"] = df.apply(
    lambda x: f"<s>[INST] {x['instruction']} [/INST] {x['output']} </s>",
    axis=1
)

# Step 3: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)


In [5]:
from huggingface_hub import login

# Login securely (input your token manually, not saved in notebook)
login()


In [6]:
!pip install -U bitsandbytes

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # required for training

# Enable 4-bit quantization for model loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load Mistral model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Enable gradient checkpointing (for memory optimization during finetune)
base_model.gradient_checkpointing_enable()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
import json
from datasets import Dataset

# Load the JSONL file manually
data_path = "/content/drive/MyDrive/qna_finetune.jsonl"

# Read the lines as JSON objects
with open(data_path, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f]

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(records)

# Preview
print(dataset[0])


{'instruction': "Are there any really good tutorials explaining branching and merging with Apache Subversion? \nAll the better if it's specific to TortoiseSVN client.", 'output': 'Version Control with Subversion\nA very good resource for source control in general. Not really TortoiseSVN specific, though.'}


In [9]:
def format_prompt(example):
    return {
        "text": f"<s>[INST] {example['instruction']} [/INST] {example['output']} </s>"
    }

formatted_dataset = dataset.map(format_prompt)


Map:   0%|          | 0/2994 [00:00<?, ? examples/s]

In [10]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/2994 [00:00<?, ? examples/s]

In [11]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare the model for training (adds cast and layer norms, etc.)
model = prepare_model_for_kbit_training(base_model)

# Define LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # these are attention projections in Mistral
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Inject LoRA adapters into the model
model = get_peft_model(model, lora_config)


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/mistral-lora-stackbot",  # Save output here
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)


In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipython-input-14-1529947149.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*ar

Step,Training Loss
10,2.095700
20,2.002200
30,2.124700
40,2.043400
50,2.016400
60,2.001800
70,2.112200
80,2.049500
90,2.005800
100,1.890000


TrainOutput(global_step=749, training_loss=1.7661101044577177, metrics={'train_runtime': 7182.7012, 'train_samples_per_second': 0.417, 'train_steps_per_second': 0.104, 'total_flos': 6.543212389230182e+16, 'train_loss': 1.7661101044577177, 'epoch': 1.0})

In [15]:
trainer.save_model("/content/drive/MyDrive/mistral-final")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-final")


('/content/drive/MyDrive/mistral-final/tokenizer_config.json',
 '/content/drive/MyDrive/mistral-final/special_tokens_map.json',
 '/content/drive/MyDrive/mistral-final/tokenizer.model',
 '/content/drive/MyDrive/mistral-final/added_tokens.json',
 '/content/drive/MyDrive/mistral-final/tokenizer.json')

In [16]:
model.save_pretrained("/content/drive/MyDrive/mistral-lora-adapter")


In [17]:
!pip install bitsandbytes
!pip install accelerate==0.28.0 transformers==4.41.0 peft==0.10.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.8.1
    Uninstalling accelerate-1.8.1:
      Successfully uninstalled accelerate-1.8.1
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninsta

In [18]:
!pip install bitsandbytes
!pip install transformers==4.40.2 peft==0.10.0 accelerate==0.28.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [19]:
import json

with open("/content/drive/MyDrive/mistral-lora-adapter/adapter_config.json") as f:
    config = json.load(f)

print(config)


{'alpha_pattern': {}, 'auto_mapping': None, 'base_model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'bias': 'none', 'corda_config': None, 'eva_config': None, 'exclude_modules': None, 'fan_in_fan_out': False, 'inference_mode': True, 'init_lora_weights': True, 'layer_replication': None, 'layers_pattern': None, 'layers_to_transform': None, 'loftq_config': {}, 'lora_alpha': 32, 'lora_bias': False, 'lora_dropout': 0.05, 'megatron_config': None, 'megatron_core': 'megatron.core', 'modules_to_save': None, 'peft_type': 'LORA', 'r': 8, 'rank_pattern': {}, 'revision': None, 'target_modules': ['q_proj', 'v_proj'], 'task_type': 'CAUSAL_LM', 'trainable_token_indices': None, 'use_dora': False, 'use_rslora': False}


In [20]:
import json

adapter_path = "/content/drive/MyDrive/mistral-lora-adapter/adapter_config.json"

# Load and fix the config
with open(adapter_path) as f:
    config = json.load(f)

# Remove the problematic key
config.pop("corda_config", None)
config.pop("eva_config", None)  # optional cleanup
config.pop("megatron_config", None)  # optional cleanup

# Save it back
with open(adapter_path, "w") as f:
    json.dump(config, f, indent=2)

print("Fixed config saved.")


Fixed config saved.


In [21]:
import json

clean_config = {
    "peft_type": "LORA",
    "base_model_name_or_path": "mistralai/Mistral-7B-v0.1",
    "r": 8,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": ["v_proj", "q_proj"],
    "inference_mode": True
}

with open("/content/drive/MyDrive/mistral-lora-adapter/adapter_config.json", "w") as f:
    json.dump(clean_config, f, indent=2)

print("Cleaned LoRA adapter config saved.")


Cleaned LoRA adapter config saved.


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

peft_config = PeftConfig.from_pretrained("/content/drive/MyDrive/mistral-lora-adapter")

base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/mistral-lora-adapter")
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)


PackageNotFoundError: No package metadata was found for bitsandbytes

In [5]:
prompt = "I'd like to create a spring bean that holds the value of a double.Something like:\n<bean id=\"doubleValue\" value=\"3.7\"/"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


NameError: name 'tokenizer' is not defined

In [8]:
!git clone https://github.com/Manas-Bhutada/Stackbot.git
%cd Stackbot


Cloning into 'Stackbot'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 40 (delta 6), reused 40 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 1.61 MiB | 5.83 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/Stackbot
